# Code for the topic modelling

Tuto - https://www.youtube.com/watch?v=arMTVXEbV9Q

## Import packages

In [4]:
# nltk
import nltk
# gensim
import gensim
import gensim.corpora as corpora
# spacy
import spacy
#vis
import pyLDAvis
import pyLDAvis.gensim

In [17]:
import pandas as pd

## Import data

In [ ]:
db_init = pd.read_csv("C:/Users/valentin.stuhlfauth/OneDrive - univ-lyon2.fr/2_2025-DUT systematic review/2_Topic modelling/0_code/2025_Pol4PED_systematic_review/2025-02-27_Screening_result.csv",sep=';',encoding='ISO-8859-1')

In [14]:
db_init["Approach"].unique()

array(['Conceptual', 'Methodological', 'Qualitative', nan, 'Reviews',
       'Quantitative', 'Modelling', 'Ingineering'], dtype=object)

In [72]:
db_init=db_init.reset_index()

In [42]:
df_text = pd.DataFrame()
df_text["text_topic"] = data["Article Title"] + " " + data["Abstract"]
df_text = df_text["text_topic"]

## Pretreatment

### Lemmatisation, generation and bigrams

In [44]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [46]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

In [47]:
data_words = gen_words(lemmatization(df_text))

In [49]:
#BIGRAMS AND TRIGRAMS
## threshold (float, optional) – Represent a score threshold for forming the phrases (higher means fewer phrases) (init = 100)
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=10)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=10)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0])

['theory', 'paper_analyze', 'relationship', 'low_carbon_city', 'climate_change', 'background', 'global_warming', 'low_carbon_city', 'construction', 'important', 'measure', 'mitigate_climate_change', 'crisis', 'low_carbon_urban', 'planning', 'key', 'technology', 'low_carbon_urban', 'construction', 'research', 'aspect', 'low_carbon_urban', 'planning', 'theory', 'concept', 'practice', 'low', 'zero', 'carbon', 'community', 'low_carbon_city', 'development', 'future', 'sum', 'low_carbon_urban', 'planning', 'research', 'progress', 'domestic', 'overseas', 'put', 'future', 'trend', 'low_carbon_urban', 'planning', 'research', 'aspect', 'demarcation', 'path', 'low_carbon_city', 'low_carbon_urban', 'planning', 'construction', 'popularity', 'low_carbon_technology', 'enforcement', 'mechanism', 'innovation', 'low_carbon_city', 'construction']


### TF-IDF (play with hypervalue)

In [ ]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.02
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [ ]:
list(dict.fromkeys(words))

### Manual Thesaurus

In [ ]:
# Stop words 
stop_words=[]
stop_words.append(['CO2'])

print(stop_words)

In [ ]:
# Stop words suppression 
data_bigrams_trigrams_stop = [[mot for mot in doc if not mot in stop_words] for doc in data_bigrams_trigrams]

## Topic modelling global

### Dictionary

In [50]:
from gensim.corpora import Dictionary
dico = Dictionary(data_bigrams_trigrams)
print(dico)

Dictionary<8958 unique tokens: ['aspect', 'background', 'carbon', 'climate_change', 'community']...>


In [51]:
id2word = corpora.Dictionary(data_bigrams_trigrams)

corpus = []
for text in data_bigrams_trigrams:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 4), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 5), (19, 1)]
aspect


### LDA (init passes=10)

In [62]:
# lissing topic appartenance = alpha (close to 0 = crisp (binary) / more than 10 = lissing of the results) (deffault = 1/num_topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=50,
                                           #random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=5,
                                           alpha="auto")

In [59]:
lda_model.print_topics()[0:2]

[(5,
  '0.000*"outgrow" + 0.000*"superlinear" + 0.000*"definite" + 0.000*"implant" + 0.000*"implantation" + 0.000*"protective" + 0.000*"allometric" + 0.000*"explored" + 0.000*"restrain" + 0.000*"commissioning"'),
 (13,
  '0.000*"outgrow" + 0.000*"superlinear" + 0.000*"definite" + 0.000*"implant" + 0.000*"implantation" + 0.000*"protective" + 0.000*"allometric" + 0.000*"explored" + 0.000*"restrain" + 0.000*"commissioning"')]

In [60]:
lda_model.show_topic(0)

[('approach', 0.054836888),
 ('project', 0.052121513),
 ('transition', 0.046698086),
 ('process', 0.044073105),
 ('sector', 0.042188846),
 ('community', 0.042035833),
 ('framework', 0.027259815),
 ('urban_agglomeration', 0.026877375),
 ('sustainable', 0.024459409),
 ('implementation', 0.023843277)]

In [56]:
# serach terms in topic
lda_model.get_term_topics("sharing", minimum_probability=0)

[(10, 0.0028584623)]

### Vizualizing the Data

In [63]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
46     0.173424  0.479382       1        1  19.832009
33     0.315221  0.433515       2        1  16.117361
49    -0.135918  0.506754       3        1  12.274782
34    -0.004414  0.513325       4        1   8.918855
39     0.500927  0.053820       5        1   4.633777
29     0.496753  0.201790       6        1   3.969357
42     0.080386  0.453429       7        1   3.439493
5     -0.501357 -0.137622       8        1   2.384051
30    -0.441444  0.257678       9        1   2.042690
47     0.246622  0.329235      10        1   1.825564
32     0.385413  0.214469      11        1   1.760898
15    -0.133965  0.433032      12        1   1.727808
38    -0.455499  0.124204      13        1   1.483051
0     -0.257560  0.345550      14        1   1.388392
24     0.285635  0.228356      15        1   1.198290
1     -0.350306  0.319606      16        1   1.190658
6     -0.137547  0.241115      17        1   1.062210
20    -0.469150  0.009089      18        1   1.060030
2      0.008449  0.279163      19        1   1.001314
26     0.318056  0.065915      20        1   0.940618
9     -0.321988  0.156949      21        1   0.936056
16     0.057017 -0.481084      22        1   0.896160
17     0.113041  0.170972      23        1   0.775330
45    -0.267790 -0.414322      24        1   0.707117
22    -0.382684 -0.165933      25        1   0.676545
7      0.380961 -0.039190      26        1   0.643116
37     0.374722 -0.285049      27        1   0.597319
18    -0.305069 -0.230105      28        1   0.587281
43     0.433163 -0.179806      29        1   0.550477
10    -0.083401 -0.471090      30        1   0.533025
23    -0.265887 -0.314052      31        1   0.502683
27     0.192289 -0.436239      32        1   0.495416
3     -0.323105 -0.028253      33        1   0.471307
4      0.191179 -0.344313      34        1   0.415130
40     0.256724 -0.274753      35        1   0.414945
41     0.209513 -0.150537      36        1   0.404052
19     0.291277 -0.112826      37        1   0.395109
21     0.047309 -0.316205      38        1   0.333216
14    -0.120240 -0.339742      39        1   0.303975
13    -0.076061 -0.254597      40        1   0.271756
48    -0.199811 -0.093620      41        1   0.234980
8     -0.110760  0.023009      42        1   0.216106
11     0.074764 -0.144769      43        1   0.142990
31    -0.030476 -0.143348      44        1   0.111405
36     0.011292 -0.028501      45        1   0.073677
44    -0.041568 -0.082888      46        1   0.027003
12    -0.007035 -0.092878      47        1   0.008197
25    -0.007036 -0.092879      48        1   0.008154
28    -0.007035 -0.092878      49        1   0.008153
35    -0.007034 -0.092878      50        1   0.008111, topic_info=              Term        Freq        Total Category  logprob  loglift
93           urban  4801.00000  4801.000000  Default  30.0000  30.0000
697       emission  1898.00000  1898.000000  Default  29.0000  29.0000
102         energy  2137.00000  2137.000000  Default  28.0000  28.0000
9      development  2546.00000  2546.000000  Default  27.0000  27.0000
55      low_carbon  1423.00000  1423.000000  Default  26.0000  26.0000
..             ...         ...          ...      ...      ...      ...
25   paper_analyze     0.00195    50.228347  Topic50  -9.1003  -0.7368
26            path     0.00195   139.902000  Topic50  -9.1003  -1.7612
27        planning     0.00195   748.665359  Topic50  -9.1003  -3.4385
28      popularity     0.00195     4.075083  Topic50  -9.1003   1.7749
29        practice     0.00195   165.198524  Topic50  -9.1003  -1.9274

[2381 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
120       2  0.990845             about
1387     13  0.973056             above
1468     17  0.974036  above_conclusion
349      38  0.866105           absence
1002      4  0.9627

## Topic modelling "Scale and Resources" > 

## Data analysis

In [64]:
# description of topics' space
doc_topics = lda_model.get_document_topics(corpus[:-1])
print(doc_topics)

In [65]:
# "sparce" matrix transformation
from gensim.matutils import corpus2csc
mat_sparce = corpus2csc(doc_topics)

print(mat_sparce)

  (5, 0)	0.026408571749925613
  (6, 0)	0.017804166302084923
  (10, 0)	0.08102951943874359
  (15, 0)	0.028569821268320084
  (17, 0)	0.014475645497441292
  (21, 0)	0.013427573256194592
  (26, 0)	0.16010437905788422
  (29, 0)	0.018302064388990402
  (32, 0)	0.015604554675519466
  (33, 0)	0.05355236306786537
  (34, 0)	0.05800873786211014
  (39, 0)	0.03783108666539192
  (42, 0)	0.08864591270685196
  (46, 0)	0.19876927137374878
  (47, 0)	0.017536601051688194
  (49, 0)	0.10699625313282013
  (5, 1)	0.04468901455402374
  (6, 1)	0.041244301944971085
  (13, 1)	0.021645143628120422
  (15, 1)	0.023561712354421616
  (18, 1)	0.013892151415348053
  (21, 1)	0.013131385669112206
  (24, 1)	0.04393310099840164
  (26, 1)	0.06432025879621506
  (29, 1)	0.03627924993634224
  :	:
  (20, 2025)	0.015533776953816414
  (24, 2025)	0.011222349479794502
  (27, 2025)	0.014651557430624962
  (29, 2025)	0.12063784152269363
  (33, 2025)	0.20882399380207062
  (34, 2025)	0.023475809022784233
  (38, 2025)	0.02848108299076557


In [66]:
# data frame
mat_dt = mat_sparce.T.toarray()
mat_dt

array([[0.        , 0.        , 0.        , ..., 0.0175366 , 0.        ,
        0.10699625],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.08628862],
       [0.        , 0.        , 0.02039475, ..., 0.        , 0.        ,
        0.20102744],
       ...,
       [0.02747214, 0.01926056, 0.        , ..., 0.01365835, 0.        ,
        0.21855922],
       [0.02381448, 0.        , 0.        , ..., 0.05243335, 0.        ,
        0.14668371],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.15873881]])

In [74]:
# data frame with polarities
dfTopic = pd.DataFrame(mat_dt, columns = ["T" + str(i) for i in range(mat_dt.shape[1])])
df_Topic_treatment = dfTopic
df_Topic_treatment['id'] = db_init['index']

print(df_Topic_treatment.head())

    T0        T1        T2   T3   T4        T5        T6   T7        T8   T9  \
0  0.0  0.000000  0.000000  0.0  0.0  0.026409  0.017804  0.0  0.000000  0.0   
1  0.0  0.000000  0.000000  0.0  0.0  0.044689  0.041244  0.0  0.000000  0.0   
2  0.0  0.000000  0.020395  0.0  0.0  0.028749  0.000000  0.0  0.097484  0.0   
3  0.0  0.000000  0.000000  0.0  0.0  0.010393  0.000000  0.0  0.000000  0.0   
4  0.0  0.067151  0.000000  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0   

   ...  T41       T42  T43  T44       T45       T46       T47  T48       T49  \
0  ...  0.0  0.088646  0.0  0.0  0.000000  0.198769  0.017537  0.0  0.106996   
1  ...  0.0  0.054272  0.0  0.0  0.017556  0.205746  0.000000  0.0  0.086289   
2  ...  0.0  0.016196  0.0  0.0  0.000000  0.262996  0.000000  0.0  0.201027   
3  ...  0.0  0.015630  0.0  0.0  0.000000  0.135810  0.000000  0.0  0.265583   
4  ...  0.0  0.000000  0.0  0.0  0.000000  0.125462  0.000000  0.0  0.047965   

   id  
0   0  
1   1  
2   2  
3   3 